In [ ]:
 

import numpy as np
import pandas as pd
import random 
from random import randint
import time

import os
import json
from pathlib import Path

import tensorflow.keras as keras
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
#from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
#from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt
from matplotlib import colors
import pandas as pd

 



submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='output_id')

sub_list=[]
for output_id in submission.index:
    task_id = output_id.split('_')[0]
    pair_id = int(output_id.split('_')[1])
    f = str(test_path / str(task_id + '.json'))
    with open(f, 'r') as read_file:
        task = json.load(read_file)
        sub_list.append(task)

sub_list

 
 ---
 ---

### 1. Create Train - Test - Split

def tts(input_list):
    Xs_test, ys_test, Xs_train, ys_train = [], [], [], []

    for task in input_list:
        X_test, y_test, X_train, y_train = [], [], [], []

        for pair in task["test"]:
            X_test.append(pair["input"])
#             y_test.append(pair["output"])      ### to be predicted !!!

        for pair in task["train"]:
            X_train.append(pair["input"])
            y_train.append(pair["output"])

        Xs_test.append(X_test)
        ys_test.append(y_test)
        Xs_train.append(X_train)
        ys_train.append(y_train)
    return Xs_test, ys_test, Xs_train, ys_train

 
#### Create origanl sized Train-Test-Split for reference 

Xs_test_orig, ys_test_orig, Xs_train_orig, ys_train_orig = tts(sub_list)

len(Xs_train_orig), len(ys_train_orig), len(Xs_test_orig), len(ys_test_orig)

len(Xs_train_orig[1]), len(ys_train_orig[1]), len(Xs_test_orig[1]), len(ys_test_orig[1])

 
#### Create actual Train-Test-Split


Xs_test, ys_test, Xs_train, ys_train = tts(sub_list)

len(Xs_train), len(ys_train), len(Xs_test), len(ys_test)

len(Xs_train[1]), len(ys_train[1]), len(Xs_test[1]), len(ys_test[1])

## Data Augmentation of the training data

# #### color
# def color_change(arr):
#     x = arr*3
# #     x = x/(x.max()+0.1)
#     x = np.interp(x, (0.3, 2.7), (0, 0.9))
#     x = np.ndarray.round(x,1)
#     return x



def augm(X_list, y_list):
    idx = 1
    X_list_new = []
    y_list_new = []
    
    for task_X, task_y in zip(X_list, y_list):
        task_li_x = []
        task_li_y = []
#         print("TASK",idx)
        length = len(Xs_train_orig[idx-1])
#         print(length)
        max_li=[]
        for i in range(length):
            max_a = max(len(task_X[i]),len(task_X[i][0]))
            max_b = max(len(task_y[i]),len(task_y[i][0]))
            max_c = max(len(Xs_test[idx-1][0]),len(Xs_test[idx-1][0][0]))
#             max_d = max(len(ys_test[idx-1][0]),len(ys_test[idx-1][0][0]))
            max_abcd = max(max_a,max_b, max_c)
            max_li.append(max_abcd)
#             print(f"max_a = max({len(task_X[i])},{len(task_X[i][0])})")
#             print(f"max_b = max({len(task_y[i])},{len(task_y[i][0])})")
#             print(max_int)
        max_int = max(max_li)
            
        for e_x,e_y in zip(task_X, task_y):
            e_x = (np.array(e_x))/10.     
            e_y = (np.array(e_y))/10.
#             print(e_x.shape,e_y.shape, max_int)
            e_x = np.pad(e_x, [(0, max_int-e_x.shape[0]), (0, max_int-e_x.shape[1])], mode='constant')
            e_y = np.pad(e_y, [(0, max_int-e_y.shape[0]), (0, max_int-e_y.shape[1])], mode='constant')
#             print(e_x.shape,e_y.shape, max_int, "\n")
            task_li_x.append(e_x.reshape(e_x.shape + (1,) ))
            task_li_y.append(e_y.reshape(e_y.shape + (1,) ))
        # ========================
         
        while len(task_li_x) < 500*length:    ###################
#             print(len(task_li_x))
            """
            choose a random method and append new array
            """
            i = randint(0,length-1) 
            new_x = (np.array(task_X[i]))/10.
#             print((new_x))
            new_y = (np.array(task_y[i]))/10.
            
            
            times = randint(1,5)
#             print(f"LENGTH: {len(task_X)} // INDEX: {i} // TIMES: {times}" )
            for j in range(times):
                
                method = random.choice([np.fliplr, np.flipud, np.rot90])#,color_change])
                new_x = method(new_x)
                new_y = method(new_y)

            new_x = np.pad(new_x, [(0, max_int-new_x.shape[0]), (0, max_int-new_x.shape[1])], mode='constant')

            new_y = np.pad(new_y, [(0, max_int-new_y.shape[0]), (0, max_int-new_y.shape[1])], mode='constant')

            
            task_li_x.append(new_x.reshape(new_x.shape + (1,) ))
            task_li_y.append(new_y.reshape(new_y.shape + (1,) ))
 
        X_list_new.append(np.array(task_li_x))
        y_list_new.append(np.array(task_li_y))

        idx +=1
    return     X_list_new, y_list_new

Xs_train_aug, ys_train_aug = augm(Xs_train, ys_train)





#### format test data

def t2l(X_list):
    li_x =[]
#     li_y =[]
    idx = 0

    for e_x in X_list:
        e_x = np.array(e_x[0])
#         e_y = np.array(e_y)
        max_train = Xs_train_aug[idx].shape[1]
        max_int = max(max(e_x.shape),max_train)
        print(e_x.shape, max_int)

        e_x = np.pad(e_x, [(0, max_int-e_x.shape[0]), (0, max_int-e_x.shape[1])], mode='constant')
        e_x = e_x / 10.
        e_x = e_x.reshape(e_x.shape + (1,) )
#             
#         e_y = np.pad(e_y, [(0, max_int-e_y.shape[0]), (0, max_int-e_y.shape[1])], mode='constant')
#         e_y = e_y / 10.
#         e_y = e_y.reshape(e_y.shape + (1,) )
# #             print(e_x.shape, e_y.shape, "\n")
        li_x.append(e_x)
#         li_y.append(e_y)

#         lenght = len(Xs_train_aug[idx])
        
#         li_x.append([np.array(li2_x)])
#         li_y.append([np.array(li2_y)])

        idx += 1
    return li_x



## Model comp

class Autoencoder(Model):
    def __init__(self, latent_dim, inp_shape, outp_shape, flat_shape):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.inp_shape = inp_shape
        self.outp_shape = outp_shape
        self.flat_shape = flat_shape
        
        
        self.encoder = Sequential([
            layers.Conv2D(160, (3, 3), activation='relu', padding='same', input_shape = inp_shape), #### 
            layers.MaxPooling2D((2, 2), padding='same'),
            layers.Conv2D(320, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2), padding='same'),
            layers.Flatten(),
            layers.Dense(latent_dim, activation='relu'),
        ])
        self.decoder = Sequential([
            layers.Dense(flat_shape, activation='sigmoid'),
            layers.Reshape(outp_shape),
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

K.clear_session()  
latent_dim = 500 
idx = 0
start = time.time()
pred_train_li = []
pred_test_li = []

for X_train, y_train, X_test, y_test in zip(Xs_train_aug, ys_train_aug, Xs_test_aug, ys_test_aug):
    K.clear_session()  
#     print(f"TASK {idx}")
    set_no = 0


    ### get input and output shapes to pass to the model
    inp_shape = X_train[0].shape
#         print('in', inp_shape)
    outp_shape = y_train[0].shape#[0]*y_set_train[0].shape[1]
#     print('out',outp_shape)
    flat_shape = outp_shape[0]*outp_shape[1]
#         print('flat',flat_shape)   
        
        
    verb = 0
#     if idx%10 == 0: verb=1    
    
    K.clear_session() 
    base_model = Autoencoder(latent_dim, inp_shape, outp_shape, flat_shape)

    base_model.compile(optimizer='adam', 
                              loss='mean_squared_error',
                              metrics=['accuracy'])

    callback = keras.callbacks.EarlyStopping(monitor='accuracy',
                                                 patience=5)

    base_model.fit(X_train, y_train,
                          epochs=100,
                          batch_size=5000,
                          #shuffle=True,
                          verbose=verb,
                          callbacks=[callback])#,
#                           validation_data=(X_set_test, y_set_test))        
        
    encoded_imgs_train = base_model.encoder(X_train[:1]).numpy()
    decoded_imgs_train = base_model.decoder(encoded_imgs_train).numpy()

    pred_train = (decoded_imgs_train.reshape(outp_shape[0:2]))*10
    
    encoded_imgs_test = base_model.encoder(np.array(Xs_test_aug[idx]).reshape((1,)+outp_shape)).numpy()
    decoded_imgs_test = base_model.decoder(encoded_imgs_test).numpy()

    pred_test = (decoded_imgs_test.reshape(outp_shape[0:2]))*10
    
    pred_train_li.append(pred_train)
    pred_test_li.append(pred_test) 
        
    idx += 1
    
end = time.time()
 
print("Total time: " + str(np.round(end - start, 1)) + " s" + "\n")


# base_model.encoder.summary()

# base_model.decoder.summary()

li=[]
for e in pred_test_li:
    x = np.rint(e)
    x = str(x).replace('.','')
    x = x.replace(' ','')
    x = x.replace('\n','')
    x = x.replace('[[', '|')
    x = x.replace('][', '|')
    x = x.replace(']]', '|')
#     x_li = x.tolist()
#     x_li = flattener(x_li)
    li.append(x)


pd_series = pd.Series(li)
# pd_series

# submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='output_id')
# submission

submission = submission.reset_index()
# submission

del submission['output']

submission['output'] = pd_series

submission= submission.set_index("output_id")

submission.to_csv('my_submission.csv')



